In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('D:/project/homecredit/installments_payments.csv')

# 依各人的時間排序，讓表格比較好閱讀，不重要的還款暫定還款金額小於預計還款金額的2%
df2 = df.sort_values(by=['SK_ID_CURR','SK_ID_PREV','NUM_INSTALMENT_NUMBER','DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT'])

In [2]:
df2['DAYS_ENTRY_DIFF'] = df2['DAYS_ENTRY_PAYMENT']-df2['DAYS_INSTALMENT'] # 新增延遲還款天數欄位，正為遲還，負為早還
df2['AMT_PAY_DIFF'] = df2['AMT_PAYMENT']-df2['AMT_INSTALMENT'] # 新增還款金額不足欄位，正為多還，負為少還
df2['DELAY'] = (df2['DAYS_ENTRY_DIFF']>0).replace(True,1)
df2['A_P/I'] = df2['AMT_PAYMENT']/df2['AMT_INSTALMENT']
pd.reset_option('all')
df2[df2['A_P/I']<0.02] # 假設這些都是不重要的還款
df2.loc[df2['A_P/I']<0.02,'DAYS_ENTRY_DIFF'] = 0 # 不重要的還款設為準時還款

C:\Users\User\AppData\Local\Temp\ipykernel_4344\1225382312.py:5: FutureWarning: column_space is deprecated and will be removed in a future version. Use df.to_string(col_space=...) instead.
  pd.reset_option('all')
C:\Users\User\AppData\Local\Temp\ipykernel_4344\1225382312.py:5: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  pd.reset_option('all')
C:\Users\User\AppData\Local\Temp\ipykernel_4344\1225382312.py:5: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  pd.reset_option('all')


In [3]:
# 依每筆貸款，整理出每期還款的最大延遲天數作為該期代表與是否延遲
df2_1 = pd.DataFrame(df2.groupby(['SK_ID_CURR','SK_ID_PREV','NUM_INSTALMENT_NUMBER']).agg({'DAYS_ENTRY_PAYMENT':['max','min'],
                                                                                                 'DAYS_ENTRY_DIFF':'max',
                                                                                                 'AMT_PAYMENT':['median','max'],
                                                                                                 'DELAY':'max'}))


df2_1.columns = ['DAYS_ENTRY_PAYMENT_MAX','DAYS_ENTRY_PAYMENT_MIN','DAYS_ENTRY_DIFF','AMT_PAYMENT_MEDIAN','AMT_PAYMENT_MAX','DELAY']

pd.reset_option('all')

df2_1.reset_index(level='NUM_INSTALMENT_NUMBER',inplace=True) # 將其中一欄index改成表格資料
df2_1.rename(columns={'NUM_INSTALMENT_NUMBER':'NUM_INSTALMENT_NUMBER_GROUP'},inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_4344\438434282.py:10: FutureWarning: column_space is deprecated and will be removed in a future version. Use df.to_string(col_space=...) instead.
  pd.reset_option('all')
C:\Users\User\AppData\Local\Temp\ipykernel_4344\438434282.py:10: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  pd.reset_option('all')
C:\Users\User\AppData\Local\Temp\ipykernel_4344\438434282.py:10: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  pd.reset_option('all')


df2_2 依各人的時間 (NUM_INSTALMENT_NUMBER) 排序

In [4]:
df2_2 = pd.DataFrame(df2_1.groupby(['SK_ID_CURR','SK_ID_PREV']).agg({'NUM_INSTALMENT_NUMBER_GROUP':['max','min'],
                                                                           'DAYS_ENTRY_PAYMENT_MAX':'max',
                                                                           'DAYS_ENTRY_PAYMENT_MIN':'min',
                                                                           'DAYS_ENTRY_DIFF':['max','mean'],
                                                                           'AMT_PAYMENT_MEDIAN':'median',
                                                                           'AMT_PAYMENT_MAX':'max',
                                                                           'DELAY':'sum'}))

# 整理出每筆貸款分成幾次還與延遲次數
df2_2.columns = ['NUM_INSTALMENT_NUMBER_GROUP_MAX',
                    'NUM_INSTALMENT_NUMBER_GROUP_MIN',
                    'DAYS_ENTRY_PAYMENT_MAX',
                    'DAYS_ENTRY_PAYMENT_MIN',
                    'DAYS_ENTRY_DIFF_MAX',
                    'DAYS_ENTRY_DIFF_MEAN',
                    'AMT_PAYMENT_MEDIAN',
                    'AMT_PAYMENT_MAX',
                    'DELAY_SUM']

df2_2.reset_index(level=('SK_ID_CURR','SK_ID_PREV'),inplace=True)
df2_2['DELAY%'] = df2_2['DELAY_SUM']/(df2_2['NUM_INSTALMENT_NUMBER_GROUP_MAX']-df2_2['NUM_INSTALMENT_NUMBER_GROUP_MIN']+1)

df2_3 排除有 NA 的列

In [5]:
df2_3 = df2.dropna(axis=0,inplace=False).copy() # 先排除有NA的列

# 先把一期貸款分多次償還的狀況整理好
# 各期與各版本的繳款狀況均看最晚的，做為評估違約的可能性
df2_3 = pd.DataFrame(df2_3.groupby(['SK_ID_CURR',
                                        'SK_ID_PREV',
                                        'NUM_INSTALMENT_NUMBER',
                                        'NUM_INSTALMENT_VERSION']).agg({'DAYS_INSTALMENT':'max',
                                                                        'DAYS_ENTRY_PAYMENT':'max',
                                                                        'AMT_INSTALMENT':'mean',
                                                                       'AMT_PAYMENT':'sum'}))

df2_3.reset_index(level=('SK_ID_CURR','SK_ID_PREV','NUM_INSTALMENT_NUMBER','NUM_INSTALMENT_VERSION'),inplace=True)
# 把另一種記帳型態整理好
df2_3 = pd.DataFrame(df2_3.groupby(['SK_ID_CURR',
                                        'SK_ID_PREV',
                                        'NUM_INSTALMENT_NUMBER']).agg({'DAYS_INSTALMENT':'max',
                                                                       'DAYS_ENTRY_PAYMENT':'max',
                                                                       'AMT_INSTALMENT':'sum',
                                                                       'AMT_PAYMENT':'mean'}))

df2_3.reset_index(level=('SK_ID_CURR','SK_ID_PREV','NUM_INSTALMENT_NUMBER'),inplace=True)
df2_3['1_DELAY'] = (df2_3['NUM_INSTALMENT_NUMBER']==1 & 
                        (df2_3['DAYS_INSTALMENT']<df2_3['DAYS_ENTRY_PAYMENT'])).replace(True,1)
df2_3['1_gauge'] = (df2_3['NUM_INSTALMENT_NUMBER']==1).replace(True,1) # 判斷此列是否為第1列
df2_3['1_gauge'].replace(0,np.nan,inplace=True)
df2_3['1_DAYS_ENTRY_DIFF'] = df2_3['DAYS_ENTRY_PAYMENT']-df2_3['DAYS_INSTALMENT'] # 遲繳天數
df2_3['1_DAYS_ENTRY_DIFF'] = df2_3['1_DAYS_ENTRY_DIFF']*df2_3['1_gauge'] # 將非第一列的遲繳天數改na

df2_3['3_DELAY_tmp1'] = df2_3['NUM_INSTALMENT_NUMBER']<=3 # 條件判斷欄，是否為前三期
df2_3['3_DELAY_tmp2'] = df2_3['DAYS_INSTALMENT']<df2_3['DAYS_ENTRY_PAYMENT']  # 條件判斷欄，是否遲繳
df2_3['3_DELAY'] = (df2_3['3_DELAY_tmp1'] & df2_3['3_DELAY_tmp2']).replace(True,1) # 綜合前兩個條件
df2_3['3_gauge'] = (df2_3['NUM_INSTALMENT_NUMBER']<=3).replace(True,1) # 判斷此列是否為前三期
df2_3['3_gauge'].replace(0,np.nan,inplace=True)
df2_3['3_DAYS_ENTRY_DIFF'] = df2_3['DAYS_ENTRY_PAYMENT']-df2_3['DAYS_INSTALMENT'] # 遲繳天數
df2_3['3_DAYS_ENTRY_DIFF'] = df2_3['3_DAYS_ENTRY_DIFF']*df2_3['3_gauge'] # 將非前三列的遲繳天數改na

df2_3['6_DELAY_tmp1'] = df2_3['NUM_INSTALMENT_NUMBER']<=6 # 新增前六期的狀況欄位，方法同上
df2_3['6_DELAY_tmp2'] = df2_3['DAYS_INSTALMENT']<df2_3['DAYS_ENTRY_PAYMENT']
df2_3['6_DELAY'] = (df2_3['6_DELAY_tmp1'] & df2_3['6_DELAY_tmp2']).replace(True,1)
df2_3['6_gauge'] = (df2_3['NUM_INSTALMENT_NUMBER']<=6).replace(True,1)
df2_3['6_gauge'].replace(0,np.nan,inplace=True)
df2_3['6_DAYS_ENTRY_DIFF'] = df2_3['DAYS_ENTRY_PAYMENT']-df2_3['DAYS_INSTALMENT'] 
df2_3['6_DAYS_ENTRY_DIFF'] = df2_3['6_DAYS_ENTRY_DIFF']*df2_3['6_gauge'] 

df2_3['9_DELAY_tmp1'] = df2_3['NUM_INSTALMENT_NUMBER']<=9 # 新增前九期的狀況欄位，方法同上
df2_3['9_DELAY_tmp2'] = df2_3['DAYS_INSTALMENT']<df2_3['DAYS_ENTRY_PAYMENT']
df2_3['9_DELAY'] = (df2_3['9_DELAY_tmp1'] & df2_3['9_DELAY_tmp2']).replace(True,1)
df2_3['9_gauge'] = (df2_3['NUM_INSTALMENT_NUMBER']<=9).replace(True,1)
df2_3['9_gauge'].replace(0,np.nan,inplace=True)
df2_3['9_DAYS_ENTRY_DIFF'] = df2_3['DAYS_ENTRY_PAYMENT']-df2_3['DAYS_INSTALMENT']
df2_3['9_DAYS_ENTRY_DIFF'] = df2_3['9_DAYS_ENTRY_DIFF']*df2_3['9_gauge']

df2_3.drop(columns=['1_gauge','3_gauge','3_DELAY_tmp1','3_DELAY_tmp2',
                       '6_gauge','6_DELAY_tmp1','6_DELAY_tmp2',
                       '9_gauge','9_DELAY_tmp1','9_DELAY_tmp2'],inplace=True)

df2_3 = pd.merge(df2_3,df2_2.iloc[:,1:3])

df2_3['-1_DELAY'] = ((df2_3['NUM_INSTALMENT_NUMBER']==df2_3['NUM_INSTALMENT_NUMBER_GROUP_MAX']) & 
                        (df2_3['DAYS_INSTALMENT']<df2_3['DAYS_ENTRY_PAYMENT'])).replace(True,1)
df2_3['-1_gauge'] = (df2_3['NUM_INSTALMENT_NUMBER']==df2_3['NUM_INSTALMENT_NUMBER_GROUP_MAX']).replace(True,1) # 判斷此列是否為最後1列
df2_3['-1_gauge'].replace(0,np.nan,inplace=True)
df2_3['-1_DAYS_ENTRY_DIFF'] = df2_3['DAYS_ENTRY_PAYMENT']-df2_3['DAYS_INSTALMENT'] # 遲繳天數
df2_3['-1_DAYS_ENTRY_DIFF'] = df2_3['-1_DAYS_ENTRY_DIFF']*df2_3['-1_gauge'] # 將非第一列的遲繳天數改na

df2_3['-3_DELAY_tmp1'] = df2_3['NUM_INSTALMENT_NUMBER']>df2_3['NUM_INSTALMENT_NUMBER_GROUP_MAX']-3 # 條件判斷欄，是否為最後三期
df2_3['-3_DELAY_tmp2'] = df2_3['DAYS_INSTALMENT']<df2_3['DAYS_ENTRY_PAYMENT']  # 條件判斷欄，是否遲繳
df2_3['-3_DELAY'] = (df2_3['-3_DELAY_tmp1'] & df2_3['-3_DELAY_tmp2']).replace(True,1) # 綜合前兩個條件
df2_3['-3_gauge'] = (df2_3['NUM_INSTALMENT_NUMBER']>df2_3['NUM_INSTALMENT_NUMBER_GROUP_MAX']-3).replace(True,1) # 判斷此列是否為最後三期
df2_3['-3_gauge'].replace(0,np.nan,inplace=True)
df2_3['-3_DAYS_ENTRY_DIFF'] = df2_3['DAYS_ENTRY_PAYMENT']-df2_3['DAYS_INSTALMENT'] # 遲繳天數
df2_3['-3_DAYS_ENTRY_DIFF'] = df2_3['-3_DAYS_ENTRY_DIFF']*df2_3['-3_gauge'] # 將非最後三列的遲繳天數改na

df2_3['-6_DELAY_tmp1'] = df2_3['NUM_INSTALMENT_NUMBER']>df2_3['NUM_INSTALMENT_NUMBER_GROUP_MAX']-6 # 新增最後六期的狀況欄位，方法同上
df2_3['-6_DELAY_tmp2'] = df2_3['DAYS_INSTALMENT']<df2_3['DAYS_ENTRY_PAYMENT']
df2_3['-6_DELAY'] = (df2_3['-6_DELAY_tmp1'] & df2_3['-6_DELAY_tmp2']).replace(True,1)
df2_3['-6_gauge'] = (df2_3['NUM_INSTALMENT_NUMBER']>df2_3['NUM_INSTALMENT_NUMBER_GROUP_MAX']-6).replace(True,1)
df2_3['-6_gauge'].replace(0,np.nan,inplace=True)
df2_3['-6_DAYS_ENTRY_DIFF'] = df2_3['DAYS_ENTRY_PAYMENT']-df2_3['DAYS_INSTALMENT'] 
df2_3['-6_DAYS_ENTRY_DIFF'] = df2_3['-6_DAYS_ENTRY_DIFF']*df2_3['-6_gauge'] 

df2_3['-9_DELAY_tmp1'] = df2_3['NUM_INSTALMENT_NUMBER']>df2_3['NUM_INSTALMENT_NUMBER_GROUP_MAX']-9 # 新增最後九期的狀況欄位，方法同上
df2_3['-9_DELAY_tmp2'] = df2_3['DAYS_INSTALMENT']<df2_3['DAYS_ENTRY_PAYMENT']
df2_3['-9_DELAY'] = (df2_3['-9_DELAY_tmp1'] & df2_3['-9_DELAY_tmp2']).replace(True,1)
df2_3['-9_gauge'] = (df2_3['NUM_INSTALMENT_NUMBER']>df2_3['NUM_INSTALMENT_NUMBER_GROUP_MAX']-9).replace(True,1)
df2_3['-9_gauge'].replace(0,np.nan,inplace=True)
df2_3['-9_DAYS_ENTRY_DIFF'] = df2_3['DAYS_ENTRY_PAYMENT']-df2_3['DAYS_INSTALMENT']
df2_3['-9_DAYS_ENTRY_DIFF'] = df2_3['-9_DAYS_ENTRY_DIFF']*df2_3['-9_gauge']

df2_3.drop(columns=['-1_gauge','-3_gauge','-3_DELAY_tmp1','-3_DELAY_tmp2',
                       '-6_gauge','-6_DELAY_tmp1','-6_DELAY_tmp2',
                       '-9_gauge','-9_DELAY_tmp1','-9_DELAY_tmp2'],inplace=True)

df2_3 = pd.DataFrame(df2_3.groupby(['SK_ID_CURR',
                                        'SK_ID_PREV']).agg({'AMT_INSTALMENT':'sum',
                                                            'AMT_PAYMENT':'sum',
                                                            '1_DELAY':'sum',
                                                            '1_DAYS_ENTRY_DIFF':'max',
                                                            '3_DELAY':'sum',
                                                            '3_DAYS_ENTRY_DIFF':['max','mean'],
                                                            '6_DELAY':'sum',
                                                            '6_DAYS_ENTRY_DIFF':['max','mean'],
                                                            '9_DELAY':'sum',
                                                            '9_DAYS_ENTRY_DIFF':['max','mean'],
                                                            '-1_DELAY':'sum',
                                                            '-1_DAYS_ENTRY_DIFF':'max',
                                                            '-3_DELAY':'sum',
                                                            '-3_DAYS_ENTRY_DIFF':['max','mean'],
                                                            '-6_DELAY':'sum',
                                                            '-6_DAYS_ENTRY_DIFF':['max','mean'],
                                                            '-9_DELAY':'sum',
                                                            '-9_DAYS_ENTRY_DIFF':['max','mean']}))

df2_3.columns = ['AMT_INSTALMENT','AMT_PAYMENT',
                    '1_DELAY_SUM','1_DAYS_ENTRY_DIFF_MAX',
                    '3_DELAY_SUM','3_DAYS_ENTRY_DIFF_MAX','3_DAYS_ENTRY_DIFF_MEAN',
                    '6_DELAY_SUM','6_DAYS_ENTRY_DIFF_MAX','6_DAYS_ENTRY_DIFF_MEAN',
                    '9_DELAY_SUM','9_DAYS_ENTRY_DIFF_MAX','9_DAYS_ENTRY_DIFF_MEAN',
                    '-1_DELAY_SUM','-1_DAYS_ENTRY_DIFF_MAX',
                    '-3_DELAY_SUM','-3_DAYS_ENTRY_DIFF_MAX','-3_DAYS_ENTRY_DIFF_MEAN',
                    '-6_DELAY_SUM','-6_DAYS_ENTRY_DIFF_MAX','-6_DAYS_ENTRY_DIFF_MEAN',
                    '-9_DELAY_SUM','-9_DAYS_ENTRY_DIFF_MAX','-9_DAYS_ENTRY_DIFF_MEAN']
df2_3['FINISHED'] = (df2_3['AMT_INSTALMENT'] == df2_3['AMT_PAYMENT']).replace(True,1)
df2_3['OUTSTANDING'] = df2_3['AMT_INSTALMENT'] - df2_3['AMT_PAYMENT'] # 新增一欄為欠款金額

df2_3.reset_index(level=('SK_ID_CURR','SK_ID_PREV'),inplace=True)

df2_3['FINISHED'] = (df2_3['OUTSTANDING']<1).replace(True,1) # 將FINISHED欄改成貸款未還小於1元皆為1

pd.reset_option('all')

df2_3.rename(columns={'AMT_INSTALMENT':'AMT_INSTALMENT_ALL','AMT_PAYMENT':'AMT_PAYMENT_ALL'},inplace=True)
# 這兩欄為該筆貸款的預計總歸還金額、實際總歸還金額

df2_3['3_DELAY_SUM'] = df2_3['3_DELAY_SUM']/3
df2_3['6_DELAY_SUM'] = df2_3['6_DELAY_SUM']/6
df2_3['9_DELAY_SUM'] = df2_3['9_DELAY_SUM']/9
df2_3.rename(columns={'1_DELAY_SUM':'1_DELAY%','3_DELAY_SUM':'3_DELAY%',
                         '6_DELAY_SUM':'6_DELAY%','9_DELAY_SUM':'9_DELAY%'},inplace=True)

df2_3['-3_DELAY_SUM'] = df2_3['-3_DELAY_SUM']/3
df2_3['-6_DELAY_SUM'] = df2_3['-6_DELAY_SUM']/6
df2_3['-9_DELAY_SUM'] = df2_3['-9_DELAY_SUM']/9
df2_3.rename(columns={'-1_DELAY_SUM':'-1_DELAY%','-3_DELAY_SUM':'-3_DELAY%',
                         '-6_DELAY_SUM':'-6_DELAY%','-9_DELAY_SUM':'-9_DELAY%'},inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_4344\4166145624.py:131: FutureWarning: column_space is deprecated and will be removed in a future version. Use df.to_string(col_space=...) instead.
  pd.reset_option('all')
C:\Users\User\AppData\Local\Temp\ipykernel_4344\4166145624.py:131: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  pd.reset_option('all')
C:\Users\User\AppData\Local\Temp\ipykernel_4344\4166145624.py:131: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  pd.reset_option('all')


Merge to previous_application

In [6]:
pre = pd.read_csv("D:/project/homecredit/previous_application.csv")

pre_arranged = pd.merge(pre,df2_2,how='outer')
pre_arranged = pd.merge(pre_arranged,df2_3,how='outer')

# pre_arranged.to_csv('D:/project/homecredit/final/prev_instalments_ETL_v2.csv',index = False)

In [ ]:
# result = pd.merge(prev_comb_data, POS_data_1,how='outer')